In [128]:
from pathlib import Path

import numpy as np
import pandas as pd

In [129]:
data=pd.read_csv("train.csv",index_col="tripid")
test=pd.read_csv("test.csv",index_col="tripid")

In [130]:
data.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [131]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17176 entries, 189123628 to 213817296
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   additional_fare            16974 non-null  float64
 1   duration                   16974 non-null  float64
 2   meter_waiting              16974 non-null  float64
 3   meter_waiting_fare         16974 non-null  float64
 4   meter_waiting_till_pickup  16974 non-null  float64
 5   pickup_time                17176 non-null  object 
 6   drop_time                  17176 non-null  object 
 7   pick_lat                   17176 non-null  float64
 8   pick_lon                   17176 non-null  float64
 9   drop_lat                   17176 non-null  float64
 10  drop_lon                   17176 non-null  float64
 11  fare                       17039 non-null  float64
 12  label                      17176 non-null  object 
dtypes: float64(10), object(3)
memory u

In [132]:
features_df=data.drop('label',axis=1)
labels_df=data['label'].replace(['correct','incorrect'],[1,0])

In [133]:
features_df.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32
189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85
189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64
189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30
189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39


In [134]:
labels_df.head(25)

tripid
189123628    1
189125358    1
189125719    1
189127273    1
189128020    1
189129552    1
189132829    1
189135103    1
189139296    1
189138671    1
189135814    0
189140969    0
189144528    1
189147041    0
189145904    1
189146174    1
189150541    1
189153229    1
189151585    1
189157464    1
189157607    0
189157476    1
189157559    1
189159951    1
189158726    1
Name: label, dtype: int64

In [135]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score

import seaborn as sns

from sklearn.ensemble import RandomForestClassifier

RANDOM_SEED = 6

In [136]:
numeric_cols = features_df.columns[features_df.dtypes != "object"].values
not_numeric_cols =features_df.columns[features_df.dtypes == "object"].values

In [137]:
numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='median'))
])


categorical_features = not_numeric_cols
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols)
    ],
    remainder = "drop"
)

In [138]:
model = RandomForestClassifier()

In [139]:
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", model),
])

In [140]:
full_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('standard_scaler',
                                                                   StandardScaler()),
                                                                  ('simple_imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  array(['additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pick_lat',
       'pick_lon', 'drop_lat', 'drop_lon', 'fare'], dtype=object))])),
                ('estimators', RandomForestClassifier())])

In [141]:
X_train, X_eval, y_train, y_eval = train_test_split(
    features_df,
    labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=labels_df,
    random_state=RANDOM_SEED
)

In [142]:
y_train.head()

tripid
205667210    0
193277385    1
201668211    1
204263911    0
206929715    1
Name: label, dtype: int64

In [143]:
full_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('standard_scaler',
                                                                   StandardScaler()),
                                                                  ('simple_imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  array(['additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pick_lat',
       'pick_lon', 'drop_lat', 'drop_lon', 'fare'], dtype=object))])),
                ('estimators', RandomForestClassifier())])

In [144]:
preds = full_pipeline.predict_proba(X_eval)
preds

array([[0.02, 0.98],
       [0.08, 0.92],
       [0.05, 0.95],
       ...,
       [0.26, 0.74],
       [0.36, 0.64],
       [0.14, 0.86]])

In [145]:
y_preds = pd.DataFrame(
    {
        "label": preds[:, 1],
    },
    index = y_eval.index
)

In [146]:
roc_auc_score(y_eval, y_preds)

0.9093859992178334

In [147]:
full_pipeline.fit(features_df, labels_df)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('standard_scaler',
                                                                   StandardScaler()),
                                                                  ('simple_imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  array(['additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pick_lat',
       'pick_lon', 'drop_lat', 'drop_lon', 'fare'], dtype=object))])),
                ('estimators', RandomForestClassifier())])

In [148]:
test_features_df = pd.read_csv("test.csv", 
                               index_col="tripid")
submission_df = pd.read_csv("sample_submission.csv",
                            index_col="tripid")

In [149]:
test_probas = full_pipeline.predict(test_features_df)
test_probas

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [150]:
submission_df.head()

,prediction
tripid,
213284604,1
213286352,0
213293973,0
213294622,1
213298687,1


In [151]:
np.testing.assert_array_equal(test_features_df.index.values, 
                              submission_df.index.values)

submission_df["prediction"] = test_probas

submission_df.to_csv('160180f_randomforest.csv', index=True)

In [152]:
%alias head powershell -command "& {Get-Content %s -Head 10}"
%head my_submission.csv

tripid,prediction
213284604,1
213286352,1
213293973,1
213294622,1
213298687,1
213299545,1
213302332,1
213302671,1
213305594,1
